In [28]:
from mandala._next.imports import *
from mandala._next.common_imports import *

In [29]:
@op
def f(x):
    return x ** 2

@op
def g(x, y):
    return x + y

@op
def h(x):
    return x + 1

storage = Storage()

# memoizing context
with storage:
    for x in range(3):
        y = f(x)

y

AtomRef(4, hid='628...', cid='a82...')

In [30]:
with storage:
    for x in range(5):
        y = f(x)
        z = g(y, h(x))
    
    for y in range(15, 20):
        z = g(y, h(x))

In [31]:
cf = storage.cf(g); cf

ComputationFrame with 3 variable(s) (25 unique refs), 1 operation(s) (10 unique calls)
Computational graph:
    output_0 = g(y=y, x=x)

In [33]:
exp_cf = cf.expand(); exp_cf

ComputationFrame with 5 variable(s) (30 unique refs), 3 operation(s) (20 unique calls)
Computational graph:
    y = h(x=x_1)
    x = f(x=x_0)
    output_0 = g(y=y, x=x)

In [34]:
exp_cf.get_df()

Extracting tuples from the computation graph:
y = h(x=x_1)
x = f(x=x_0)
output_0 = g(x=x, y=y)...


,x_1,h,y,x_0,f,x,g,output_0
0,1,"Call(h, cid='568...', hid='239...')",2,1.0,"Call(f, cid='845...', hid='a55...')",1,"Call(g, cid='033...', hid='8e7...')",3
1,4,"Call(h, cid='b7d...', hid='b30...')",5,NaN,None,15,"Call(g, cid='19e...', hid='290...')",20
2,0,"Call(h, cid='9f2...', hid='dae...')",1,0.0,"Call(f, cid='82f...', hid='b98...')",0,"Call(g, cid='65a...', hid='ec6...')",1
3,4,"Call(h, cid='b7d...', hid='b30...')",5,NaN,None,16,"Call(g, cid='0a8...', hid='701...')",21
4,4,"Call(h, cid='b7d...', hid='b30...')",5,NaN,None,19,"Call(g, cid='fc9...', hid='74f...')",24
5,4,"Call(h, cid='b7d...', hid='b30...')",5,4.0,"Call(f, cid='05e...', hid='f87...')",16,"Call(g, cid='0a8...', hid='52a...')",21
6,2,"Call(h, cid='b03...', hid='d83...')",3,2.0,"Call(f, cid='77b...', hid='1cb...')",4,"Call(g, cid='cdb...', hid='c95...')",7
7,4,"Call(h, cid='b7d...', hid='b30...')",5,NaN,None,18,"Call(g, cid='17e...', hid='6a5...')",23
8,3,"Call(h, cid='148...', hid='d1d...')",4,3.0,"Call(f, cid='f41...', hid='dab...')",9,"Call(g, cid='602...', hid='d68...')",13
9,4,"Call(h, cid='b7d...', hid='b30...')",5,NaN,None,17,"Call(g, cid='ae0...', hid='2ad...')",22


In [ ]:
sess.dump()

In [ ]:
restricted_cf['x'].refs

In [ ]:
with storage:
    for x in range(5):
        y = f(x)
        if storage.unwrap(y) > 5:
            z = g(x, y)

In [ ]:
storage.cf(f).\
  expand().\
    eval()

In [ ]:
df.rename(columns={'output_0_0': 'output_1'})

In [ ]:
cf = storage.cf(g)
cf.expand(inplace=True)
cf.eval()

In [ ]:
cf.delete_calls()

In [ ]:
@op
def avg_items(xs: MList[int]) -> float:
    return sum(xs) / len(xs)

@op
def get_xs(n: int) -> MList[int]:
    return list(range(n))

storage = Storage()

with storage:
    xs = get_xs(10)
    for i in range(2, 10, 2):
        avg = avg_items(xs[:i])

In [ ]:
cf = storage.cf(avg_items).expand()
cf.eval()

In [ ]:
storage = Storage()

@op
def inc(eggs):
    print(f'Executing inc with eggs={eggs}')
    return eggs + 1

with storage:
    spam = inc(eggs=42)

@op
def make_breakfast(spam, eggs):
    print(f'Executing make_breakfast with spam={spam} and eggs={eggs}')
    return spam + eggs

with storage:
    eggs = 42
    spam = inc(eggs=eggs)
    more_spam = inc(eggs=spam)
    breakfast = make_breakfast(spam=spam, eggs=eggs)
    breakfast_2 = make_breakfast(spam=more_spam, eggs=eggs)


cf = storage.cf(make_breakfast)
cf

cf = cf.expand()
cf.eval()

In [ ]:
storage = Storage()

### a few simple ops covering all cases we care about
@op
def f(x: int) -> int:
    return x + 1

@op
def g(x: int, y: int) -> Tuple[int, int]: # multiple inputs and outputs
    return x + 2, y ** 2

@op 
def inc_list(things: MList[int]) -> MList[int]: # lists as inputs and outputs
    return [x + 1 for x in things]

@op
def inc_dict(things: MDict[str, int]) -> MDict[str, int]: # dicts as inputs and outputs
    return {k: v + 1 for k, v in things.items()}

@op
def fancy_signature(x: int, *args, y: bool = False, **kwargs) -> Tuple[int, List[int], MList[int]]: # fancy signatures
    return x, [int(a) for a in args], [int(v) for v in kwargs.values()]

In [ ]:
### simple workflow
with storage:
    x = f(1)
    y, t = g(x, 2)
    z = inc_list([1, 2])
    z = inc_list([x, y])
    z = inc_list([1, x])
    w = inc_dict({'x': x, 'y': y})
    a = fancy_signature(1, 2, x, y=True, z=4, w=5)